In [ ]:
import os
import sys
from typing import List

import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
from src.data.requests_downloader import UrlDownloader
from src.features.dataset import Dataset
from src.features.metrics import CustomMeanIoU
from src.models.model_builder import Model
from src.visualization.visualize import PredictionMasks
from src.features.evaluation_utils import (
    PredictionIoU,
    ConfusionMatrix,
    HistoryUtilities,
    History
)
from src.features.loss_functions import SemanticSegmentationLoss
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
print(f"TensorFlow Version: {tf.__version__}")
print()
print(f"Python {sys.version}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

In [ ]:
VERSION = '12.1'

BATCH_SIZE = 4
IMG_HEIGHT = 512
IMG_WIDTH = 512

NUM_CLASSES = 5

In [ ]:
PROJECT_PATH = UrlDownloader().get_project_root()
if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

# Load datasets

In [ ]:
UrlDownloader().download_project_preprocessed_dataset()

In [ ]:
DATASET_PATH = os.path.join(PROJECT_PATH, 'data/processed')
landcover_dataset = Dataset(DATASET_PATH, IMG_HEIGHT, IMG_WIDTH, NUM_CLASSES, BATCH_SIZE)
train_dataset, val_dataset, test_dataset = landcover_dataset.generate_datasets()

print("Train Dataset:", train_dataset)
print("Number of images in Train Dataset:", BATCH_SIZE * len(train_dataset))
print("Val Dataset:", val_dataset)
print("Number of images in Val Dataset:", BATCH_SIZE * len(val_dataset))
print("Test Dataset:", test_dataset)
print("Number of images in Val Dataset:", BATCH_SIZE * len(test_dataset))

# Class balance 

In [ ]:
landcover_dataset.get_dataframe_of_previously_calculated_class_balance_class_balance().plot.bar(y="pixel_count", xlabel="Class name", ylabel="Count")
landcover_dataset.get_dataframe_of_previously_calculated_class_balance_class_balance()

# Evaluation utilities

In [ ]:
custom_mIoU_metric = CustomMeanIoU(num_classes=NUM_CLASSES, name='mIoU')

### Learning rate schedule [optimizer]

In [ ]:
# NUM_TRAIN_IMAGES = BATCH_SIZE * len(train_dataset)
# NUM_VAL_IMAGES = BATCH_SIZE * len(val_dataset)
#
# initial_learning_rate = 0.001
# learning_rate_decay_factor = 0.94
# steps_per_epoch = int(NUM_TRAIN_IMAGES/BATCH_SIZE)



# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#                 initial_learning_rate=initial_learning_rate,
#                 decay_steps=steps_per_epoch,
#                 decay_rate=learning_rate_decay_factor,
#                 staircase=True)

In [ ]:
initial_learning_rate = 0.005
rate_decay = 0.94

def lr_schedule(epoch):
    lr = initial_learning_rate * (rate_decay ** (epoch // 2))
    return lr

learning_rate_callback = LearningRateScheduler(lr_schedule)

### Get current lr [metric]

In [ ]:
# #  https://stackoverflow.com/questions/47490834/how-can-i-print-the-learning-rate-at-each-epoch-with-adam-optimizer-in-keras
#
# def get_lr_metric(optimizer):
#     def lr(y_true, y_pred):
#         return optimizer._decayed_lr(tf.float32)
#     return lr
#
# optimizer_adam = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
# lr_metric = get_lr_metric(optimizer_adam)

## Training settings

In [ ]:
# creating a model
WEIGHTS = "cityscapes"
SHOULD_FREEZE_LAYERS = True
LAST_LAYER_TO_FREEZE = 359  # All from the begining up to 359 will be frozen
ACTIVATION = "softmax"
DEEPLAB_VERSION = "deeplabv3plus"
OUTPUT_STRIDE = 16  # or 8

# compiling the model
OPTIMIZER = tf.keras.optimizers.Adam(
    learning_rate=initial_learning_rate,
    decay=4e-5
)
LOSS = SemanticSegmentationLoss(NUM_CLASSES).basnet_hybrid_loss
METRICS = ["accuracy", custom_mIoU_metric]

# training the model
EPOCHS = 40


## Training

 DeepLabv3+ model architecture

https://keras.io/examples/vision/deeplabv3_plus/

In [ ]:
revision = f"{VERSION}.1"
revision

In [ ]:
deeplab_model = Model(
    revision=revision,
    batch_size=BATCH_SIZE,
    input_image_height=IMG_HEIGHT,
    input_image_width=IMG_WIDTH,
    number_of_classes=NUM_CLASSES,
    pretrained_weights=WEIGHTS,
    do_freeze_layers=SHOULD_FREEZE_LAYERS,
    last_layer_frozen=LAST_LAYER_TO_FREEZE,
    activation=ACTIVATION,
    model_architecture=DEEPLAB_VERSION,
    output_stride=OUTPUT_STRIDE,
    optimizer=OPTIMIZER,
    loss_function=LOSS,
    metrics=METRICS
)
deeplab_model.save_model_revision(
        initial_learning_rate=initial_learning_rate,
        final_learning_rate=None
)

model = deeplab_model.get_deeplab_model()
model.compile(*deeplab_model.get_compile_parameters)

# tf.keras.utils.plot_model(model, to_file=data_save_dir + '/Deeplabv3plus_v8_5.jpg', show_shapes=True)

## Callbacks
### Model checkpoint

In [ ]:
def create_folders(paths: List[str]):
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)

In [ ]:
path_best_loss = PROJECT_PATH + f'/data/weights/{deeplab_model.model_architecture}_v{deeplab_model.revision}/best_loss/checkpoint'
dir_path_loss= PROJECT_PATH + f'/data/weights/{deeplab_model.model_architecture}_v{deeplab_model.revision}/best_loss'
path_best_miou = PROJECT_PATH + f'/data/weights/{deeplab_model.model_architecture}_v{deeplab_model.revision}/best_miou/checkpoint'
dir_path_miou = PROJECT_PATH + f'/data/weights/{deeplab_model.model_architecture}_v{deeplab_model.revision}/best_miou'

create_folders([
    dir_path_loss,
    dir_path_miou,
])

best_val_loss = tf.keras.callbacks.ModelCheckpoint(
    path_best_loss,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min',
    save_freq='epoch'
)

best_miou = tf.keras.callbacks.ModelCheckpoint(
    path_best_miou,
    monitor='val_mIoU',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='max',
    save_freq='epoch'
)

early_stopping_val_miou = tf.keras.callbacks.EarlyStopping(
    monitor='val_mIoU',
    patience=2,
    mode='max'
)

CALLBACKS = [
    learning_rate_callback,
    early_stopping_val_miou,
    best_val_loss,
    best_miou
]

In [ ]:
# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=40,
    callbacks=CALLBACKS,
)


## Evaluate the model

In [ ]:
model.load_weights(path_best_miou)
test_eval = model.evaluate(test_dataset)

In [ ]:
deeplab_model.update_revision_with_history_and_evaluation(history, test_eval)

In [ ]:
iou_every_class = PredictionIoU(model, test_dataset, NUM_CLASSES).get_iou_for_every_class()
iou_every_class

In [ ]:
# PredictionMasks(model, landcover_dataset, NUM_CLASSES).display_overlay_predictions_for_test_set(4, (25, 25))

### Save history to file

In [ ]:
TRAINING_DATA_PATH = PROJECT_PATH + f'/data/results/{deeplab_model.model_architecture}_v{deeplab_model.revision}'

# history_utils = HistoryUtilities()
# history_utils.dump_model_history_to_file(history, TRAINING_DATA_PATH, "history.json")

History([history]).display_history_plots(save_folder_path=TRAINING_DATA_PATH)

### Confusion matrix

In [ ]:
class_names = ['background', 'buildings', 'woodland', 'water', 'roads']

confusion_matrix = ConfusionMatrix(model, test_dataset, NUM_CLASSES)
confusion_matrix.display_confusion_matrix(class_names)

# Further training the model
## You may change the settings
E.g. Freeze less layers, change optimizer's learning rate, add metric, etc.

In [ ]:
revision = f"{VERSION}.2"
revision

In [ ]:
initial_learning_rate = 0.0001
rate_decay = 0.94

def lr_schedule(epoch):
    lr = initial_learning_rate * (rate_decay ** (epoch // 2))
    return lr

learning_rate_callback = LearningRateScheduler(lr_schedule)

OPTIMIZER = tf.keras.optimizers.Adam(
    learning_rate=initial_learning_rate,
    decay=4e-5
)

In [ ]:
deeplab_model = Model(
    revision=revision,
    batch_size=BATCH_SIZE,
    input_image_height=IMG_HEIGHT,
    input_image_width=IMG_WIDTH,
    number_of_classes=NUM_CLASSES,
    pretrained_weights=WEIGHTS,
    do_freeze_layers=SHOULD_FREEZE_LAYERS,
    last_layer_frozen=LAST_LAYER_TO_FREEZE-40,  # e.g. freeze 40 layers less
    activation=ACTIVATION,
    model_architecture=DEEPLAB_VERSION,
    output_stride=OUTPUT_STRIDE,
    optimizer=OPTIMIZER,
    loss_function=LOSS,
    metrics=METRICS
)

deeplab_model.save_model_revision(
    initial_learning_rate=initial_learning_rate,
    final_learning_rate=None
)

model = deeplab_model.get_deeplab_model()
model.compile(*deeplab_model.get_compile_parameters)
model.load_weights(path_best_miou)

# tf.keras.utils.plot_model(model, to_file=data_save_dir + '/Deeplabv3plus_v8_5.jpg', show_shapes=True)

In [ ]:
path_best_loss = PROJECT_PATH + f'/data/weights/{deeplab_model.model_architecture}_v{deeplab_model.revision}/best_loss/checkpoint'
dir_path_loss= PROJECT_PATH + f'/data/weights/{deeplab_model.model_architecture}_v{deeplab_model.revision}/best_loss'
path_best_miou = PROJECT_PATH + f'/data/weights/{deeplab_model.model_architecture}_v{deeplab_model.revision}/best_miou/checkpoint'
dir_path_miou = PROJECT_PATH + f'/data/weights/{deeplab_model.model_architecture}_v{deeplab_model.revision}/best_miou'

create_folders([
    dir_path_loss,
    dir_path_miou,
])

best_val_loss = tf.keras.callbacks.ModelCheckpoint(
    path_best_loss,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min',
    save_freq='epoch'
)

best_miou = tf.keras.callbacks.ModelCheckpoint(
    path_best_miou,
    monitor='val_mIoU',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='max',
    save_freq='epoch'
)

early_stopping_val_miou = tf.keras.callbacks.EarlyStopping(
    monitor='val_mIoU',
    patience=2,
    mode='max'
)

CALLBACKS = [
    learning_rate_callback,
    early_stopping_val_miou,
    best_val_loss,
    best_miou
]

In [ ]:
# load weights with best results
history_2 = model.fit(
    train_dataset, 
    validation_data=val_dataset, 
    epochs=EPOCHS,
    callbacks=CALLBACKS,
)

In [ ]:
path_best_miou

In [ ]:
model.load_weights(path_best_miou)
test_eval = model.evaluate(test_dataset)

In [ ]:
model.load_weights(path_best_loss)
test_eval_loss = model.evaluate(test_dataset)

In [ ]:
deeplab_model.update_revision_with_history_and_evaluation(history_2, test_eval)

# Third

In [ ]:
revision = f"{VERSION}.3"
revision

In [ ]:
initial_learning_rate = 1e-5
rate_decay = 0.90

def lr_schedule(epoch):
    lr = initial_learning_rate * (rate_decay ** (epoch // 2))
    return lr

learning_rate_callback = LearningRateScheduler(lr_schedule)

OPTIMIZER = tf.keras.optimizers.Adam(
    learning_rate=initial_learning_rate,
    decay=4e-5
)

In [ ]:
path_best_loss = PROJECT_PATH + f'/data/weights/{deeplab_model.model_architecture}_v{deeplab_model.revision}/best_loss/checkpoint'
dir_path_loss= PROJECT_PATH + f'/data/weights/{deeplab_model.model_architecture}_v{deeplab_model.revision}/best_loss'
path_best_miou = PROJECT_PATH + f'/data/weights/{deeplab_model.model_architecture}_v{deeplab_model.revision}/best_miou/checkpoint'
dir_path_miou = PROJECT_PATH + f'/data/weights/{deeplab_model.model_architecture}_v{deeplab_model.revision}/best_miou'

In [ ]:
deeplab_model = Model(
    revision=revision,
    batch_size=BATCH_SIZE,
    input_image_height=IMG_HEIGHT,
    input_image_width=IMG_WIDTH,
    number_of_classes=NUM_CLASSES,
    pretrained_weights=WEIGHTS,
    do_freeze_layers=SHOULD_FREEZE_LAYERS,
    last_layer_frozen=LAST_LAYER_TO_FREEZE-80,  # e.g. freeze 40 layers less
    activation=ACTIVATION,
    model_architecture=DEEPLAB_VERSION,
    output_stride=OUTPUT_STRIDE,
    optimizer=OPTIMIZER,
    loss_function=LOSS,
    metrics=METRICS
)

deeplab_model.save_model_revision(
    initial_learning_rate=initial_learning_rate,
    final_learning_rate=None
)

model = deeplab_model.get_deeplab_model()
model.compile(*deeplab_model.get_compile_parameters)
model.load_weights(path_best_miou)

# tf.keras.utils.plot_model(model, to_file=data_save_dir + '/Deeplabv3plus_v8_5.jpg', show_shapes=True)

In [ ]:
# create_folders([
#     dir_path_loss,
#     dir_path_miou,
# ])

best_val_loss = tf.keras.callbacks.ModelCheckpoint(
    path_best_loss,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='min',
    save_freq='epoch'
)

best_miou = tf.keras.callbacks.ModelCheckpoint(
    path_best_miou,
    monitor='val_mIoU',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='max',
    save_freq='epoch'
)

early_stopping_val_miou = tf.keras.callbacks.EarlyStopping(
    monitor='val_mIoU',
    patience=3,
    mode='max'
)

CALLBACKS = [
    learning_rate_callback,
    early_stopping_val_miou,
    best_val_loss,
    best_miou
]

In [ ]:
# load weights with best results
history_3 = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=CALLBACKS,
)

In [ ]:
path_best_miou

In [ ]:
model.load_weights(path_best_miou)
test_eval = model.evaluate(test_dataset)

In [ ]:
model.load_weights(path_best_loss)
test_eval_loss = model.evaluate(test_dataset)

In [ ]:
deeplab_model.update_revision_with_history_and_evaluation(history_3, test_eval)